# Assignement 1 study of Bachelor data from IS Academia

## useful imports

In [14]:
import pandas as pd

## Import Bachelor data

Reading the pickle with the bachelor data frame

In [15]:
bachelor_df = pd.read_pickle("bachelor")
bachelor_df.head()

Civilité  Orientation Bachelor  \
Year      Semester            Sciper                                   
2016-2017 Bachelor semestre 1 235688  Monsieur                   NaN   
                              274015  Monsieur                   NaN   
                              268410    Madame                   NaN   
                              271464  Monsieur                   NaN   
                              274518  Monsieur                   NaN   

                                      Orientation Master  Spécialisation  \
Year      Semester            Sciper                                       
2016-2017 Bachelor semestre 1 235688                 NaN             NaN   
                              274015                 NaN             NaN   
                              268410                 NaN             NaN   
                              271464                 NaN             NaN   
                              274518                 NaN             NaN   

                                     Filière opt.  Mineur   Statut  \
Year      Semester            Sciper                                 
2016-2017 Bachelor semestre 1 235688          NaN     NaN  Présent   
                              274015          NaN     NaN  Présent   
                              268410          NaN     NaN  Présent   
                              271464          NaN     NaN  Présent   
                              274518          NaN     NaN  Présent   

                                     Type Echange Ecole Echange  
Year      Semester            Sciper                             
2016-2017 Bachelor semestre 1 235688          NaN           NaN  
                              274015          NaN           NaN  
                              268410          NaN           NaN  
                              271464          NaN           NaN  
                              274518          NaN           NaN

All students that did semester 1 and 6

In [16]:
bachelor_sciper = s1.ix[s1.index & s6.index].index

In [17]:
s1 = bachelor_df.xs('Bachelor semestre 1', level=1)

s1 = s1[s1.Statut == 'Présent'].reset_index(0)

s6 = bachelor_df.xs('Bachelor semestre 6', level=1)

s6 = s6[s6.Statut == 'Présent'].reset_index(0)

bachelor_sciper = s1.ix[s1.index & s6.index].index

bachelor_s1_s6 = bachelor_df.loc[bachelor_df.index.get_level_values("Sciper").isin(bachelor_sciper.values)]

bachelor_s1_s6

Civilité  Orientation Bachelor  \
Year      Semester            Sciper                                   
2016-2017 Bachelor semestre 5 215576  Monsieur                   NaN   
                              219665    Madame                   NaN   
                              214939  Monsieur                   NaN   
                              234523  Monsieur                   NaN   
                              235493  Monsieur                   NaN   
                              217575  Monsieur                   NaN   
                              225258  Monsieur                   NaN   
                              236716  Monsieur                   NaN   
                              226647  Monsieur                   NaN   
                              223344  Monsieur                   NaN   
                              228275  Monsieur                   NaN   
                              223892  Monsieur                   NaN   
                              217098  Monsieur                   NaN   
                              214293  Monsieur                   NaN   
                              237482  Monsieur                   NaN   
                              213287  Monsieur                   NaN   
                              227359  Monsieur                   NaN   
                              224297  Monsieur                   NaN   
                              227311    Madame                   NaN   
                              204324  Monsieur                   NaN   
                              238150  Monsieur                   NaN   
                              216801    Madame                   NaN   
                              225691  Monsieur                   NaN   
                              228149    Madame                   NaN   
                              212826  Monsieur                   NaN   
                              237151  Monsieur                   NaN   
                              226652  Monsieur                   NaN   
          Bachelor semestre 6 227311    Madame                   NaN   
2015-2016 Bachelor semestre 5 202293  Monsieur                   NaN   
                              228219  Monsieur                   NaN   
...                                        ...                   ...   
2007-2008 Bachelor semestre 1 175280  Monsieur                   NaN   
          Bachelor semestre 2 169569  Monsieur                   NaN   
                              174905  Monsieur                   NaN   
                              179406  Monsieur                   NaN   
                              179428  Monsieur                   NaN   
                              179449  Monsieur                   NaN   
                              178553  Monsieur                   NaN   
                              179426  Monsieur                   NaN   
                              180731  Monsieur                   NaN   
                              171619  Monsieur                   NaN   
                              179157  Monsieur                   NaN   
                              178711  Monsieur                   NaN   
                              178786  Monsieur                   NaN   
                              178656  Monsieur                   NaN   
                              178718  Monsieur                   NaN   
                              181121  Monsieur                   NaN   
                              175379  Monsieur                   NaN   
                              178604  Monsieur                   NaN   
                              175190  Monsieur                   NaN   
                              181248  Monsieur                   NaN   
                              181244  Monsieur                   NaN   
                              169731  Monsieur                   NaN   
                              181076  Monsieur                   NaN   
      

In [112]:
idx = pd.index
bachelor_s1_s6.sort_index(level=[0,1,2], inplace=True)
bachelor_s1_s6.loc[("2016-2017",slice(None),slice(None)),:]

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Civilité  Orientation Bachelor  \
Year      Semester            Sciper                                   
2016-2017 Bachelor semestre 5 204324  Monsieur                   NaN   
                              212826  Monsieur                   NaN   
                              213287  Monsieur                   NaN   
                              214293  Monsieur                   NaN   
                              214939  Monsieur                   NaN   
                              215576  Monsieur                   NaN   
                              216801    Madame                   NaN   
                              217098  Monsieur                   NaN   
                              217575  Monsieur                   NaN   
                              219665    Madame                   NaN   
                              223344  Monsieur                   NaN   
                              223892  Monsieur                   NaN   
                              224297  Monsieur                   NaN   
                              225258  Monsieur                   NaN   
                              225691  Monsieur                   NaN   
                              226647  Monsieur                   NaN   
                              226652  Monsieur                   NaN   
                              227311    Madame                   NaN   
                              227359  Monsieur                   NaN   
                              228149    Madame                   NaN   
                              228275  Monsieur                   NaN   
                              234523  Monsieur                   NaN   
                              235493  Monsieur                   NaN   
                              236716  Monsieur                   NaN   
                              237151  Monsieur                   NaN   
                              237482  Monsieur                   NaN   
                              238150  Monsieur                   NaN   
          Bachelor semestre 6 227311    Madame                   NaN   

                                      Orientation Master  Spécialisation  \
Year      Semester            Sciper                                       
2016-2017 Bachelor semestre 5 204324                 NaN             NaN   
                              212826                 NaN             NaN   
                              213287                 NaN             NaN   
                              214293                 NaN             NaN   
                              214939                 NaN             NaN   
                              215576                 NaN             NaN   
                              216801                 NaN             NaN   
                              217098                 NaN             NaN   
                              217575                 NaN             NaN   
                              219665                 NaN             NaN   
                              223344                 NaN             NaN   
                              223892                 NaN             NaN   
                              224297                 NaN             NaN   
                              225258                 NaN             NaN   
                              225691                 NaN             NaN   
                              226647                 NaN             NaN   
                              226652                 NaN             NaN   
                              227311                 NaN             NaN   
                              227359                 NaN             NaN   
                              228149                 NaN             NaN   
                              228275                 NaN             NaN   
                              234523                 NaN             NaN   
                              235493                 

## Compute the number of semesters spent for the Bachelor

Let's find how many semesters each student spent for to get its Bachelor. To do we iterate over all the scipers and count the number of status (that includes semesters in Erasmus exchange and 'off semesters' as well):

In [151]:
bachelor_semesters = []
scipers = bachelor_s1_s6.index.get_level_values(2).drop_duplicates()

for sciper in scipers:
    student = bachelor_s1_s6.xs(sciper, level=2)
    num_sems = student['Statut'].count()
    civ = student['Civilité'][0]
    bachelor_semesters.append([sciper, civ, num_sems])

bachelor_semesters = pd.DataFrame(bachelor_semesters,columns=['Sciper', 'Civilité', 'Number of semesters'])

We now group the results by the 'Civilité' and average the number of semesters 

In [196]:
pop = bachelor_semesters.groupby(bachelor_semesters.Civilité)['Civilité'].count()
mean = bachelor_semesters.groupby(bachelor_semesters.Civilité)['Number of semesters'].mean()
pd.concat([pop,mean], axis=1, keys=['Count','Number of semesters'])

,Count,Number of semesters
Civilité,,
Madame,24,6.958333
Monsieur,273,7.249084
